In [21]:
%load_ext autoreload
%autoreload
%cd /home/axel/master/ais-visualization/src

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/axel/master/ais-visualization/src


In [22]:
from main import *

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd
import numpy as np
from pathlib import Path

In [23]:
def _parse_boosting_info(prop_values: dict, prop_type: str, enum_name: str):
    cols = {}
    for p in prop_values:
        if type(p) == str:
            if len(cols) == 0:
                continue
            else:
                for k, v in cols.items():
                    v.append(pd.NA)
        else:
            memberships: dict = p[enum_name]
            for k, v in memberships.items():
                cur_val = cols.get(k)
                if cur_val is None:
                    cols[k] = [v]
                else:
                    cols[k].append(v)

    df_dict = {}
    for k, v in cols.items():
        # v = np.add.reduceat(v, np.arange(0, len(v), 5))
        df_dict[f"{prop_type}_{k}"] = v
    print(df_dict)
    return (prop_type, df_dict)

In [24]:
def gen_cor_w_noreg_cols(df: pd.DataFrame, key: str):
    n_cor = df[f"{key}_cor"]
    n_wrong = df[f"{key}_wrong"]
    n_no_reg = df[f"{key}_no_reg"]

    df[f"{key}_precision"] = n_cor/(n_cor+n_wrong)
    df[f"{key}_accuracy"] = n_cor/(n_cor+n_wrong+n_no_reg)

In [25]:
file_pre = "ex_boosting/logg_dat_Ionosphere_boosting.json"
with open(f"./../../ais/logg_dat/{file_pre}", "r") as f:
    js_obj = json.load(f)
[(v, i["property_type"]) for i in js_obj["meta_props"] for k, v in enumerate(i) ]

metas = js_obj["meta_props"]

for idx, v in enumerate(metas):
    print(idx, v["property_type"])

boost_accuracy = metas[0]
boost_accuracy_test = metas[1]

(p, d) = _parse_boosting_info(boost_accuracy_test["prop_values"], boost_accuracy_test["property_type"], "MappedInts")
(m, l) = _parse_boosting_info(boost_accuracy["prop_values"], boost_accuracy["property_type"], "MappedInts")
df_dict = []
l.update(d)
ddf = pd.DataFrame(l)

gen_cor_w_noreg_cols(ddf, "BoostAccuracyTest")
gen_cor_w_noreg_cols(ddf, "BoostAccuracy")


0 BoostAccuracy
1 BoostAccuracyTest
2 FoldAccuracy
3 Runtime
{'BoostAccuracyTest_cor': [31, 32, 32, 32, 32, 29, 28, 28, 29, 29, 33, 34, 34, 33, 34, 32, 32, 32, 32, 32, 27, 27, 27, 27, 27, 30, 29, 29, 30, 30, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 32, 32, 31, 31, 31, 44, 43, 44, 43, 43, 32, 33, 33, 33, 33, 29, 30, 30, 29, 29, 32, 32, 33, 34, 34, 31, 31, 32, 32, 33, 29, 29, 28, 28, 29, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 32, 32, 32, 32, 32, 42, 41, 43, 43, 43, 33, 33, 33, 33, 33, 30, 31, 32, 32, 32, 33, 33, 33, 33, 33, 33, 34, 33, 33, 33, 28, 28, 28, 27, 27, 29, 29, 29, 29, 30, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 31, 32, 32, 32, 32, 44, 43, 43, 43, 43, 34, 34, 34, 34, 34, 30, 32, 31, 31, 31, 33, 32, 33, 33, 33, 32, 32, 32, 32, 32, 29, 28, 27, 27, 27, 31, 30, 30, 31, 31, 31, 32, 32, 32, 32, 34, 33, 33, 33, 33, 32, 31, 32, 31, 31, 41, 43, 43, 43, 43, 32, 33, 33, 33, 33, 31, 30, 30, 30, 29, 31, 33, 33, 33, 33, 32, 32, 32, 32, 32, 27, 28, 28, 28, 28, 29, 30, 30, 

In [26]:
i = 4
step = 5
ddf[i::step]["BoostAccuracy_accuracy"]

4      0.993691
9      0.996845
14     0.996845
19     0.993691
24     1.000000
         ...   
479    0.987382
484    0.996845
489    0.993691
494    0.993691
499    0.993464
Name: BoostAccuracy_accuracy, Length: 100, dtype: float64

In [27]:
stds_train = []
accs_train = []
stds_test = []
accs_test = []
print
step = 5
for i in range(0, 5):
    # print(df[i - 10:i:][])
    stds_train.append(ddf[i::step]["BoostAccuracy_accuracy"].std())
    accs_train.append(ddf[i::step]["BoostAccuracy_accuracy"].mean())
    stds_test.append(ddf[i::step]["BoostAccuracyTest_accuracy"].std())
    accs_test.append(ddf[i::step]["BoostAccuracyTest_accuracy"].mean())

In [28]:
fig = go.Figure()
df_t = ddf[:10:]
markers = ["circle", "square", "diamond", "cross", "star", "triangle-up", "triangle-down", "triangle-left", "triangle-right"]

def plot_boosting_train_test_acc(ddf: pd.DataFrame):
    n = 0

    fig.add_trace(go.Scatter(x=np.arange(0, len(stds_test)), y=accs_test,
                        mode='lines+markers',
                        name='BoostAccuracyTest',
                        error_y=dict(
                                     type='data',  # value of error bar given in data coordinates
                                     array=stds_test,
                                     visible=True),
                        marker=dict(
                                    symbol=markers[n],
                                    size=8,
                                    #angleref="previous",
                                        )))
    
    n+=1
    fig.add_trace(go.Scatter(x=np.arange(0, len(stds_train)), y=accs_train,
                        mode='lines+markers',
                        name='BoostAccuracyTrain',
                        error_y=dict(
                                     type='data',  # value of error bar given in data coordinates
                                     array=stds_train,
                                     visible=True),
                        marker=dict(
                                    symbol=markers[n],
                                    size=8,
                                    #angleref="previous",
                                        )))
    n += 1
    

    
    fig.update_layout(height = 500)
    fig.update_xaxes(range=[-0.5,4.5])
    fig.update_layout(legend_title_text=" Mean accuracy <br> of 5 boosting rounds")
    write_image(fig, f"{file_pre}", f"agg_boost.png")

    fig.show()
plot_boosting_train_test_acc(ddf)